# Aprendizado por Reforço

- Bruno Pilão - 202201037911
- Mateus Norcia - 202201038381
- Isabelle -


Este trabalho tem como objetivo aplicar os conceitos fundamentais de Aprendizado por Reforço (Reinforcement Learning) através da modelagem de um Processo de Decisão de Markov (MDP), utilizando o ambiente Taxi-v3 disponível na biblioteca Gymnasium.

O ambiente simula um cenário onde um táxi deve buscar um passageiro em um ponto específico e levá-lo até o destino correto, dentro de um grid (5x5) com obstáculos e paredes. O agente (táxi) aprende a agir por tentativa e erro, buscando maximizar a recompensa acumulada com base nas ações tomadas.

![Ambiente Taxi-v3](img/enviroment_taxi.png)



## 🧩 Definição do Markov Decision Process


### Estados (S):
- O ambiente tem 500 estados possíveis. Cada estado representa:
- Posição do táxi (fila, coluna)
- Local de origem do passageiro (uma das 5 localizações fixas)
    - 0: Vermelho
    - 1: Verde
    - 2: Amarelo
    - 3: Azul
    - 4: No taxi

### Ações (A):
- 0 = Mover para sul
- 1 = Mover para norte
- 2 = Mover para leste
- 3 = Mover para oeste
- 4 = Pegar passageiro
- 5 = Deixar passageiro

### Recompensas
- R(s, a) = −1 por passo.
- Estados (0,1);(0,4);(4,1);(3,4) são terminais com recompensa +20 (depedendo da Localização do Hotel).
- Estado tem recompensa negativa −10, por executar indevidamente as ações de embarque e desembarque.
### Transições (P(s'|s,a)):
Determinísticas, exceto quando o táxi tenta atravessar uma parede, nesse caso, ele permanece no mesmo estado.


In [ ]:
# Importando pacotes necessarios compilação collab
! pip install gymnasium numpy matplotlib pandas seaborn tqdm pygame -q


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

In [68]:
# Criando o ambiente Taxi-v3
env_default = gym.make("Taxi-v3")
env_raining = gym.make("Taxi-v3",is_rainy=True)

In [36]:
# Definindo estados do ambiente
estados=[
 (0, 0), (0, 1), (0, 2), (0, 3), (0, 4),
 (1, 0), (1, 1), (1, 2), (1, 3), (1, 4),
 (2, 0), (2, 1), (2, 2), (2, 3), (2, 4),
 (3, 0), (3, 1), (3, 2), (3, 3), (3, 4),
 (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)
]

#Definindo ações do ambiente
acoes = {
    0: "South ⬇️",
    1: "North ⬆️",
    2: "East ➡️",
    3: "West ⬅️",
    4: "Pickup 🚖",
    5: "Dropoff 🛑"
}

## Equação de Belman


In [ ]:
env_default.action_space.n, env_default.observation_space.n

(np.int64(6), np.int64(500))

In [92]:
state = 80
for action in range(env_default.action_space.n):
    transitions = env_default.unwrapped.P[state][action]
    print(f"Ação {action}:")
    for prob, next_state, reward, done in transitions:
        print(f" → Próximo estado: {next_state}, Recompensa: {reward}, Terminal: {done} (Probabilidade: {prob})")


Ação 0:
 → Próximo estado: 180, Recompensa: -1, Terminal: False (Probabilidade: 1.0)
Ação 1:
 → Próximo estado: 80, Recompensa: -1, Terminal: False (Probabilidade: 1.0)
Ação 2:
 → Próximo estado: 80, Recompensa: -1, Terminal: False (Probabilidade: 1.0)
Ação 3:
 → Próximo estado: 60, Recompensa: -1, Terminal: False (Probabilidade: 1.0)
Ação 4:
 → Próximo estado: 80, Recompensa: -10, Terminal: False (Probabilidade: 1.0)
Ação 5:
 → Próximo estado: 80, Recompensa: -10, Terminal: False (Probabilidade: 1.0)


## Recompensas

- -1 per step unless other reward is triggered.

- +20 delivering passenger.

- -10 executing “pickup” and “drop-off” actions illegally.